## 物件偵測的多項指標
* Rafael Padilla __***et al.***__發表統整物件偵測的多項指標文章[[Paper]](https://github.com/rafaelpadilla/Object-Detection-Metrics/raw/master/paper_survey_on_performance_metrics_for_object_detection_algorithms.pdf)，並在開源的Github[[連結]](https://github.com/rafaelpadilla/Object-Detection-Metrics#how-to-use-this-project)提供不同的物件偵測任務比賽，所用的不同的指標算法，並可以使用使用者介面[[連結]](https://github.com/rafaelpadilla/review_object_detection_metrics)直接作指標計算。

### 01. 安裝
* 第一步至Github使用者介面的存儲庫下載，左鍵點擊 **Code** → **Download ZIP**
* 強烈建議您使用此使用者介面的存儲庫中可用的文件創建**anaconda**環境。`environment.yml`要創建環境並安裝所有必需的套件，請命令提示字元移動至下載完成的存儲庫資料夾(使用`cd path`指令)，運行以下命令：
    * `conda env create -n <env_name> --file environment.yml`
    * 進入環境：`conda activate <env_name>`
    * 安裝套件：`python setup.py install`
    * 運行使用者界面：`python run.py`
    
<img src="https://raw.githubusercontent.com/rafaelpadilla/review_object_detection_metrics/main/data/images/printshot_main_screen.png" width="60%" />

### 02. 使用說明

每個紅色數字編號，如下所述：

1. **Annotations(標註)**：選擇包含實際標籤檔案的資料夾。
2. **Images(影像)**：選擇包含影像的資料夾。
3. **Classes(類別)**：YOLO (.txt) 訓練格式表示具有 ID（順序整數）的類。對於這標註類型，您需要格式為一個 txt 文件，其中每行列出一個類。第一行是 id 為 0 的類，第二行是 id 為 1 的類，以此類推。請參見此處用於表示 VOC PASCAL 數據集類的檔案範例。
4. **Coordinate formats(坐標格式)**：選擇標註檔案的格式。
5. **Ground-truth statistics(實際框統計)**：這是可選功能，統計每個Ground-Truth的邊界框之類別數量，並使用邊界框可視化圖像。要使用此選項，您必須已給予第二項的影像資料夾。
6. **Annotations(標註)**：選擇包含帶有檢測的標註檔案的資料夾。
7. **Classes(類)**：同第三項。
8. **Coordinate formats(坐標格式)**：選擇用於表示檢測的檔案之格式。
9. **Detections statistics(檢測統計)**：這是可選功能，統計每個類的檢測數量。並可藉由在影像繪製檢測框及實際框來可視化檢測的能力。
10. **Metrics(指標)**：選擇至少一個指標來評估您的檢測。對於PASCAL VOC AP和mAP，可以選擇不同的IOU。請注意，PASCAL VOC AP 指標中使用的默認 IOU 臨界值為 0.5。
11. **Output(輸出)**：選擇將保存 PASCAL VOC AP 圖的資料夾。
12. **RUN(運行)**：運行計算指標。根據數據集的數量和檢測的格式，可能需要一段時間。相對坐標中的檢測通常比其他格式需要更長的時間來讀取。

#### 備註：
* 可視化數據集的統計數據（項目5、9：實際與檢測框之統計），以確保您選擇正確的格式。如果格式不正確，這些框將在圖像上顯示不正確或無法開啟。
* 路徑不能有**中文字**。
* 標註格式的範例位於 `data\database\gts`
* 檢測格式的範例位於 `data\database\dets` 

## 儲存YOLOv4檢測的結果，格式轉為尚未正規化的左上角與寬高
* 為上述項目8的左邊第三個格式 `(*) <class_id> <confidence> <left> <top> <width> <height> (ABSOLUTE)`
* Absolute (絕對) 與 Relative (相對) 指的是原始位值(未正規化) 與 正規化後(數值介於0-1間)的位置
* 產生的檢測結果txt檔案名稱，需要**與標籤檔案及影像名稱完全相同**!!

In [7]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import os

In [8]:
net = cv2.dnn.readNet("yolov4-tiny_last.weights","yolov4-tiny.cfg")
layer_names = net.getLayerNames()
output_layers = [layer_names[i-1] for i in net.getUnconnectedOutLayers()]

In [9]:
classes = [line.strip() for line in open("coco.names")]

In [10]:
colors = [(0,0,255),(0,255,0),(255,0,0)]

In [11]:
def yolo_detect_save(img):
    height, width, channels = img.shape
    blob = cv2.dnn.blobFromImage(img, 1/255.0, (416, 416), (0, 0, 0), True, crop=False)
    net.setInput(blob)
    outs = net.forward(output_layers)
    
    boxes, confidences, class_ids = [], [], []
    predict_str = ""
    
    for out in outs:
        for detection in out:
            tx, ty, tw, th, confidence = detection[0:5]
            scores = detection[5:]
            class_id = np.argmax(scores)
            if confidence>0:
                center_x = int(tx*width)
                center_y = int(ty*height)
                w = int(tw*width)
                h = int(th*height)
                x = int(center_x-w/2)
                y = int(center_y-h/2)
                boxes.append([x, y, w, h])
                confidences.append(float(confidence))
                class_ids.append(class_id)
                
    indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.5)
    
    for i in range(len(boxes)):
        if i in indexes:
            x, y, w, h = boxes[i]
            label = class_ids[i]
            confidence = confidences[i]
            predict_str += str(label)+" "+str(confidence)+" "+str(x)+" "+str(y)+" "+str(w)+" "+str(h)+"\n"
            cv2.rectangle(img, (x, y), (x+w, y+h), colors[label], 2)
            cv2.putText(img, str(label), (x, y-5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, colors[label], 1)
    
    return predict_str, img

In [12]:
# 預檢測的影像之資料夾
path_img = "images"
# 檢測後儲存的資料夾
path_predict = "predict"
filenames = os.listdir(path_img)

for i in range(len(filenames)):
    img = cv2.imread(path_img+"/"+filenames[i])
    img = cv2.resize(img, None, fx=1.0, fy=1.0)
    predict_str, img = yolo_detect_save(img)
    filenames_no = os.path.splitext(filenames[i])[0]

    with open(path_predict+"/"+filenames_no+".txt", mode="w") as f:
        print(predict_str, file=f)